## creating ENv and downloading csv files.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
# some csv files, uncomment to download
#!wget -q https://sample-videos.com/csv/Sample-Spreadsheet-1000-rows.csv
#!wget -q https://www.stats.govt.nz/assets/Uploads/Business-operations-survey/Business-operations-survey-2022/Download-data/business-operations-survey-2022-information-and-communications-technology.csv
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dataClean").getOrCreate()
spark

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [ ]:
!mv /content/Sample-Spreadsheet-1000-rows.csv /content/ssmp.csv

In [ ]:
dfspark = spark.read\
          .format("csv")\
          .option("header","true")\
          .option("inferSchema","true")\
          .load("/content/ssmp.csv")

dfspark.show(5)

+---+------------------------------------------------+------------------+---+-------+------+-----+-------+----------------------+----+
|  1|Eldon Base for stackable storage shelf, platinum|Muhammed MacIntyre|  3|-213.25| 38.94|   35|Nunavut|Storage & Organization| 0.8|
+---+------------------------------------------------+------------------+---+-------+------+-----+-------+----------------------+----+
|  2|                            "1.7 Cubic Foot C...|      Barry French|293| 457.81|208.16|68.02|Nunavut|            Appliances|0.58|
|  3|                            Cardinal Slant-D�...|      Barry French|293|  46.71|  8.69| 2.99|Nunavut|  Binders and Binde...|0.39|
|  4|                                            R380|     Clay Rozendal|483|1198.97|195.99| 3.99|Nunavut|  Telephones and Co...|0.58|
|  5|                            Holmes HEPA Air P...|    Carlos Soltero|515|  30.94| 21.78| 5.94|Nunavut|            Appliances| 0.5|
|  6|                            G.E. Longer-Life ...| 

In [ ]:
!kaggle datasets download -d guavocado/pokemon-stats-1025-pokemons
#works. to download pokemon data from site , UNZIP IT USING BELOW COMMANDS

Dataset URL: https://www.kaggle.com/datasets/guavocado/pokemon-stats-1025-pokemons
License(s): CC0-1.0
  0% 0.00/43.8k [00:00<?, ?B/s]
100% 43.8k/43.8k [00:00<00:00, 45.7MB/s]


In [ ]:
!mv /content/pokemon-stats-1025-pokemons.zip

In [ ]:
!apt-get install unzip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!unzip /content/pokemon-stats-1025-pokemons.zip

Archive:  /content/pokemon-stats-1025-pokemons.zip
  inflating: pokemon_data.csv        


In [ ]:
#deleting zip file
!rm /content/pokemon-stats-1025-pokemons.zip

## reading data from a csv file (pokemon data file)

In [ ]:
dfs = spark.read\
          .format("csv")\
          .option("inferSchema","true")\
          .option("header","true")\
          .load("/content/pokemon_data.csv")
dfs.show(6,truncate=False)

+------+----------+----------+-----+------+--------------+------+------+--------+-----------+--------------+---+------+-------+------+------+-----+-----+--------------------+----------+---------------+--------+-----------+----------+----------+------------+--------------+----------+-------------+
|dexnum|name      |generation|type1|type2 |species       |height|weight|ability1|ability2   |hidden_ability|hp |attack|defense|sp_atk|sp_def|speed|total|ev_yield            |catch_rate|base_friendship|base_exp|growth_rate|egg_group1|egg_group2|percent_male|percent_female|egg_cycles|special_group|
+------+----------+----------+-----+------+--------------+------+------+--------+-----------+--------------+---+------+-------+------+------+-----+-----+--------------------+----------+---------------+--------+-----------+----------+----------+------------+--------------+----------+-------------+
|1     |Bulbasaur |1         |Grass|Poison|Seed Pokémon  |0.7   |6.9   |Overgrow|Chlorophyll|null         

In [ ]:
from pyspark.sql.functions import col
dfs.filter(col('type1').like("Elec%")).orderBy("attack",ascending=False).show()

+------+----------+----------+--------+--------+-------------------+------+------+--------------+--------------+--------------+---+------+-------+------+------+-----+-----+------------------+----------+---------------+--------+-----------+------------+----------+------------+--------------+----------+-------------+
|dexnum|      name|generation|   type1|   type2|            species|height|weight|      ability1|      ability2|hidden_ability| hp|attack|defense|sp_atk|sp_def|speed|total|          ev_yield|catch_rate|base_friendship|base_exp|growth_rate|  egg_group1|egg_group2|percent_male|percent_female|egg_cycles|special_group|
+------+----------+----------+--------+--------+-------------------+------+------+--------------+--------------+--------------+---+------+-------+------+------+-----+-----+------------------+----------+---------------+--------+-----------+------------+----------+------------+--------------+----------+-------------+
|   466|Electivire|         4|Electric|    null|T

In [ ]:
dfs.groupBy('special_group').count().orderBy('special_group').show()

+---------------+-----+
|  special_group|count|
+---------------+-----+
|Ancient Paradox|   10|
|   Baby Pokemon|   19|
|         Fossil|   25|
| Future Paradox|   10|
|      Legendary|   70|
|       Mythical|   23|
|       Ordinary|  857|
|    Ultra Beast|   11|
+---------------+-----+



In [ ]:
dfs.select('name','dexnum','type1','weight').filter('weight>=50.0')\
.orderBy('weight').show()

+----------+------+--------+------+
|      name|dexnum|   type1|weight|
+----------+------+--------+------+
|  Zweilous|   634|    Dark|  50.0|
| Dragapult|   887|  Dragon|  50.0|
|Hitmonchan|   107|Fighting|  50.2|
|  Skarmory|   227|   Steel|  50.5|
|   Darkrai|   491|    Dark|  50.5|
|    Snover|   459|   Grass|  50.5|
|      Sawk|   539|Fighting|  51.0|
|     Klang|   600|   Steel|  51.0|
|    Cacnea|   331|   Grass|  51.3|
|   Yanmega|   469|     Bug|  51.5|
|   Gallade|   475| Psychic|  52.0|
|  Blaziken|   257|    Fire|  52.0|
|  Sceptile|   254|   Grass|  52.2|
|  Clamperl|   366|   Water|  52.5|
|   Seviper|   336|  Poison|  52.5|
|    Zapdos|   145|Electric|  52.6|
| Aegislash|   681|   Steel|  53.0|
|   Lucario|   448|Fighting|  54.0|
| Poliwrath|    62|   Water|  54.0|
| Heracross|   214|     Bug|  54.0|
+----------+------+--------+------+
only showing top 20 rows



In [ ]:
lis=["Fighting","Fire","Steel"]
dfs.filter((dfs['generation']>5) & (dfs['type1'].isin(lis)) & (dfs['name'].like("%i%"))).show()

+------+------------+----------+--------+------+-------------------+------+------+-------------+-----------+--------------+---+------+-------+------+------+-----+-----+--------------------+----------+---------------+--------+-----------+------------+----------+------------+--------------+----------+-------------+
|dexnum|        name|generation|   type1| type2|            species|height|weight|     ability1|   ability2|hidden_ability| hp|attack|defense|sp_atk|sp_def|speed|total|            ev_yield|catch_rate|base_friendship|base_exp|growth_rate|  egg_group1|egg_group2|percent_male|percent_female|egg_cycles|special_group|
+------+------------+----------+--------+------+-------------------+------+------+-------------+-----------+--------------+---+------+-------+------+------+-----+-----+--------------------+----------+---------------+--------+-----------+------------+----------+------------+--------------+----------+-------------+
|   653|    Fennekin|         6|    Fire|  null|       

In [ ]:
dfs.groupBy('type1').sum('attack','defense','hp').orderBy('type1').show()

+--------+-----------+------------+-------+
|   type1|sum(attack)|sum(defense)|sum(hp)|
+--------+-----------+------------+-------+
|     Bug|       5607|        5775|   4689|
|    Dark|       3960|        3223|   3411|
|  Dragon|       3662|        2783|   2945|
|Electric|       4305|        3664|   3822|
|   Fairy|       1938|        2058|   2070|
|Fighting|       4108|        2894|   3043|
|    Fire|       5452|        4411|   4609|
|  Flying|        719|         594|    630|
|   Ghost|       2401|        2642|   2163|
|   Grass|       7658|        7302|   6774|
|  Ground|       3736|        3436|   2979|
|     Ice|       2307|        2288|   2347|
|  Normal|       8777|        7063|   9148|
|  Poison|       3173|        3059|   2964|
| Psychic|       3780|        4021|   4307|
|    Rock|       5015|        5690|   3962|
|   Steel|       3168|        3999|   2580|
|   Water|       9694|        9418|   9496|
+--------+-----------+------------+-------+



In [ ]:
#to find the total attach power of all pokemons
dfs.agg({'attack':'sum'}).show()

+-----------+
|sum(attack)|
+-----------+
|      79460|
+-----------+



In [ ]:
dfs.groupBy('type1').max().orderBy('type1').show()

+--------+-----------+---------------+-----------+-----------+-------+-----------+------------+-----------+-----------+----------+----------+---------------+-----------------+-------------------+
|   type1|max(dexnum)|max(generation)|max(height)|max(weight)|max(hp)|max(attack)|max(defense)|max(sp_atk)|max(sp_def)|max(speed)|max(total)|max(catch_rate)|max(percent_male)|max(percent_female)|
+--------+-----------+---------------+-----------+-----------+-------+-----------+------------+-----------+-----------+----------+----------+---------------+-----------------+-------------------+
|     Bug|        988|              9|        3.2|      333.6|    107|        139|         230|        145|        230|       160|       600|            255|            100.0|              100.0|
|    Dark|       1004|              9|        5.8|      888.0|    223|        135|         125|        135|        135|       135|       680|            255|            100.0|              100.0|
|  Dragon|       100

In [ ]:
dfs.groupBy("type1").count().orderBy("count").show()

+--------+-----+
|   type1|count|
+--------+-----+
|  Flying|    9|
|   Fairy|   29|
|     Ice|   31|
|   Ghost|   35|
|   Steel|   36|
|  Dragon|   37|
|  Ground|   40|
|Fighting|   40|
|  Poison|   42|
|    Dark|   45|
|    Rock|   58|
|Electric|   59|
| Psychic|   60|
|    Fire|   66|
|     Bug|   83|
|   Grass|  103|
|  Normal|  118|
|   Water|  134|
+--------+-----+



In [ ]:
dfs.select('name','type1','species','hp','attack').filter((dfs['hp']>=69) & (dfs['attack']>55)).orderBy('name').show()

+----------+--------+--------------------+---+------+
|      name|   type1|             species| hp|attack|
+----------+--------+--------------------+---+------+
| Abomasnow|   Grass|  Frost Tree Pokémon| 90|    92|
|  Accelgor|     Bug|   Shell Out Pokémon| 80|    70|
|Aerodactyl|    Rock|      Fossil Pokémon| 80|   105|
|    Aggron|   Steel|  Iron Armor Pokémon| 70|   110|
| Alomomola|   Water|      Caring Pokémon|165|    75|
|   Altaria|  Dragon|     Humming Pokémon| 75|    70|
|    Amaura|    Rock|      Tundra Pokémon| 77|    59|
|   Ambipom|  Normal|   Long Tail Pokémon| 75|   100|
| Amoonguss|   Grass|    Mushroom Pokémon|114|    85|
|  Ampharos|Electric|       Light Pokémon| 90|    75|
|Annihilape|Fighting| Rage Monkey Pokémon|110|   115|
|  Appletun|   Grass|Apple Nectar Pokémon|110|    85|
|  Arboliva|   Grass|       Olive Pokémon| 78|    69|
|  Arcanine|    Fire|   Legendary Pokémon| 90|   110|
|    Arceus|  Normal|       Alpha Pokémon|120|   120|
|Archaludon|   Steel|       

In [ ]:
dfs.groupBy("type1").sum().show()

In [ ]:
dfs.printSchema()

root
 |-- dexnum: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- generation: integer (nullable = true)
 |-- type1: string (nullable = true)
 |-- type2: string (nullable = true)
 |-- species: string (nullable = true)
 |-- height: double (nullable = true)
 |-- weight: double (nullable = true)
 |-- ability1: string (nullable = true)
 |-- ability2: string (nullable = true)
 |-- hidden_ability: string (nullable = true)
 |-- hp: integer (nullable = true)
 |-- attack: integer (nullable = true)
 |-- defense: integer (nullable = true)
 |-- sp_atk: integer (nullable = true)
 |-- sp_def: integer (nullable = true)
 |-- speed: integer (nullable = true)
 |-- total: integer (nullable = true)
 |-- ev_yield: string (nullable = true)
 |-- catch_rate: integer (nullable = true)
 |-- base_friendship: string (nullable = true)
 |-- base_exp: string (nullable = true)
 |-- growth_rate: string (nullable = true)
 |-- egg_group1: string (nullable = true)
 |-- egg_group2: string (nullable = tru

In [ ]:
!touch hello.sh

In [ ]:
!bash hello.sh

hello World
this is my bash code
